In [ ]:
import geopandas as gpd
import numpy as np
import pandas as pd
import plotly.express as px
import spei as si
from scipy.stats import fisk, gamma, norm
from tqdm.auto import tqdm

In [ ]:
spain_map = gpd.read_file("../data/maps/spain-map.geojson")

In [ ]:
spain_map.head()

In [ ]:
df = pd.read_csv(
    "../data/climate-data/Spain_Avg_Water_Balance.csv", parse_dates=["date"]
)

In [ ]:
df.head()

In [ ]:
df.isna().sum()

In [ ]:
df = df.dropna()

In [ ]:
df.dtypes

In [ ]:
dfs_list = []

for province_name, province_data in tqdm(df.groupby(by="province_name")):
    province_data = province_data.sort_values(by="date").set_index("date")
    province_data["spei"] = si.spei(series=province_data["mean"], dist=fisk)
    dfs_list.append(province_data)

In [ ]:
dfs_list[0]

In [ ]:
all_data = pd.concat(dfs_list)

In [ ]:
all_data = all_data.reset_index()

In [ ]:
SPEI_base = pd.read_csv(
    "../data/climate-data/SPEI_by_province.csv", parse_dates=["date"]
)

In [ ]:
SPEI_base.head()

In [ ]:
SPEI_base.dtypes

# Compare data

In [ ]:
all_data["province_name"].nunique()

In [ ]:
SPEI_base["province"].nunique()

In [ ]:
compare_df = (
    pd.merge(
        left=all_data,
        right=SPEI_base,
        left_on=["date", "province_name"],
        right_on=["date", "province"],
        how="inner",
    )
    .rename(columns={"spei": "terra_climate_spei", "SPEI": "SPEIbase"})
    .drop(columns=["mean", "province"])
)

In [ ]:
compare_df.head()

In [ ]:
compare_df["province_name"].unique()

In [ ]:
sevilla_data = compare_df.loc[compare_df["province_name"] == "Sevilla"]

In [ ]:
sevilla_data["abs_diff"] = np.abs(
    sevilla_data["terra_climate_spei"] - sevilla_data["SPEIbase"]
)

In [ ]:
fig = px.line(data_frame=sevilla_data, x="date", y="abs_diff")

fig.show()

In [ ]:
fig = px.line(data_frame=sevilla_data, x="date", y=["terra_climate_spei", "SPEIbase"])

fig.show()

In [ ]:
all_data.to_csv('../data/climate-data/terra-climate-spei.csv', index=False)